# Level 1: Rice Crop Discovery Tool Benchmark Notebook

In [1]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization
import ipyleaflet
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns
import multiprocessing as mp
import itertools
# Data Science
import numpy as np
import pandas as pd

# Feature Engineering
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score,classification_report,confusion_matrix

# Planetary Computer Tools
import pystac
import pystac_client
import odc
from pystac_client import Client
from pystac.extensions.eo import EOExtension as eo
from odc.stac import stac_load
import xarray as xr
import planetary_computer as pc
import yaml

with open("api_key.yml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)

# Pass your API key here
pc.settings.set_subscription_key(cfg['planetary_computer']['key'])
# Others
import requests
import rich.table
from itertools import cycle
from tqdm import tqdm
tqdm.pandas()

## Response Variable

Before building the model, we need to load in the rice crop presence data. We have curated for you data from a certain region in Vietnam for the year 2020. The data consists of  geo locations (Latitude and Longitude) with a tag specifying if the crop present in each geo location is rice or not.  

In [2]:
crop_presence_data = pd.read_csv("./Data/Crop_Location_Data_20221201.csv")
crop_presence_data.head()

,Latitude and Longitude,Class of Land
0,"(10.323727047081501, 105.2516346045924)",Rice
1,"(10.322364360592521, 105.27843410554115)",Rice
2,"(10.321455902933202, 105.25254306225168)",Rice
3,"(10.324181275911162, 105.25118037576274)",Rice
4,"(10.324635504740822, 105.27389181724476)",Rice


## Predictor Variables

<p align ="justify">Now that we have our crop location data, it is time to gather the predictor variables from the Sentinel-1 dataset. For a more in-depth look regarding the Sentinel-1 dataset and how to query it, see the Sentinel-1 <a href="https://challenge.ey.com/api/v1/storage/admin-files/6403146221623637-63ca8d537b1fe300146c79d0-Sentinel%201%20Phenology.ipynb/"> supplementary 
notebook</a>.
    

<p align = "justify">Sentinel-1 radar data penetrates through the clouds, thus helping us to get the band values with minimal atmospheric attenuation. Band values such as VV and VH help us in distinguishing between the rice and non rice crops. Hence we are choosing VV and VH as predictor variables for this experiment. 
        
<ul>
<li>VV - gamma naught values of signal transmitted with vertical polarization and received with vertical polarization with radiometric terrain correction applied.

<li>VH - gamma naught values of signal transmitted with vertical polarization and received with horizontal polarization with radiometric terrain correction applied.
</ul>

<h4 style="color:rgb(195, 52, 235)"><strong>Tip 1</strong></h4>
<p align="justify">Participants might explore other combinations of bands from the Sentinel-1 data. For example, you can use mathematical combinations of bands to generate various <a href="https://challenge.ey.com/api/v1/storage/admin-files/3868217534768359-63ca8dc8aea56e00146e3489-Comprehensive%20Guide%20-%20Satellite%20Data.docx">vegetation indices </a> which can then be used as features in your model.


### Accessing the Sentinel-1 Data

<p align = "Justify">To get the Sentinel-1 data, we write a function called <i><b>get_sentinel_data.</b></i> This function will fetch VV and VH band values for a particular location over the specified time window. In this example, we have extracted VV and VH values for a day (21st March 2020). </p>

<h4 style="color:rgb(195, 52, 235)"><strong>Tip 2</strong></h4>
<p align="justify"> Extract VV and VH band values for an entire year. Different land classes (e.g., agriculture, water, urban) will have different annual variability. This variability will be better than a single date for accurately identifying land classes. Please find below a demonstration of extracting data for a day (21st March 2020).

In [3]:
def get_sentinel_data(latlong,time_slice= "2020-01-01/2020-01-31",assets=['vv','vh']):
    '''
    Returns VV and VH values for a given latitude and longitude 
    Attributes:
    latlong - A tuple with 2 elements - latitude and longitude
    time_slice - Timeframe for which the VV and VH values have to be extracted
    assets - A list of bands to be extracted
    '''

    latlong=[float(i) for i in latlong.replace('(','').replace(')','').replace(' ','').split(',')]
    bbox_of_interest = (float(latlong[1]) , float(latlong[0]), float(latlong[1]) , float(latlong[0]))
    time_of_interest = time_slice
    catalog = pystac_client.Client.open(
            "https://planetarycomputer.microsoft.com/api/stac/v1",
    )
    search = catalog.search(
        collections=["sentinel-1-rtc"], bbox=bbox_of_interest, datetime=time_of_interest
    )
    items = list(search.get_all_items())
    data = stac_load(items, patch_url=pc.sign,bands=["vv", "vh"], lat=(latlong[0], latlong[0]), lon=(latlong[1], latlong[1])).to_dataframe().reset_index()
    data['lat_actual'],data['lon_actual'] = latlong
    return data

<h4 style="color:rgb(195, 52, 235)"><strong>Tip 3 </strong></h4>

Explore the approach of building a bounding box (e.g., 5x5 pixels) around the given latitude and longitude positions and then extract the aggregated band values (e.g., average, median) to get normalized band values to build the model. Radar data has inherent variability at the pixel level due to variable scattering response from the target. This effect is called “speckle” and it is common to filter the data to smooth these variations. Try using a 3x3, 5x5 or 7x7 window around the specific latitude and longitude point to get improved results.

In [4]:
## Function call to extract VV,VH Values
fetch_data = True # Set to False if you have already extracted the data
if fetch_data:
    time_slice = "2020-01-01/2020-12-31"
    vh_vv=[]
    pool = mp.Pool(mp.cpu_count()-10)
    inputs = zip(crop_presence_data['Latitude and Longitude'].values, [time_slice]*len(crop_presence_data))
    vh_vv_df = pool.starmap(get_sentinel_data, tqdm(inputs,total=len(crop_presence_data)))
    pool.close()
    pool.join()
    df = pd.concat(vh_vv_df)
    df.to_csv('./Data/sentinel_data.csv',index=False)
else:
    df = pd.read_csv('./Data/sentinel_data.csv')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 600/600 [00:00<00:00, 36604.30it/s]


,y,x,time,spatial_ref,vv,vh,lat_actual,lon_actual
0,1141215.0,527555.0,2020-01-02 22:45:22.476685,32648,0.147335,0.027236,10.323727,105.251635
1,1141215.0,527555.0,2020-01-09 11:11:41.420743,32648,0.100532,0.035662,10.323727,105.251635
2,1141215.0,527555.0,2020-01-14 22:45:21.932565,32648,0.072585,0.028481,10.323727,105.251635
3,1141215.0,527555.0,2020-01-20 22:45:54.277524,32648,0.129160,0.025950,10.323727,105.251635
4,1141215.0,527555.0,2020-01-21 11:11:40.981936,32648,0.118973,0.039736,10.323727,105.251635
...,...,...,...,...,...,...,...,...
74,1106825.0,573825.0,2020-12-09 22:46:02.163664,32648,0.341642,0.072926,10.012126,105.673613
75,1106825.0,573825.0,2020-12-10 11:11:48.884291,32648,0.197114,0.048743,10.012126,105.673613
76,1106825.0,573825.0,2020-12-15 22:45:29.514434,32648,0.315534,0.056959,10.012126,105.673613
77,1106825.0,573825.0,2020-12-21 22:46:01.606092,32648,0.195670,0.071512,10.012126,105.673613


# Submission

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display,clear_output
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
print(tf.config.list_physical_devices('GPU'))

2023-03-26 10:05:34.577875: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-26 10:05:34.742746: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-26 10:05:34.774246: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-26 10:05:35.295589: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


### Expermient 1 : VV Dataset

##### Data Fetch

In [11]:
crop_presence_data = pd.read_csv("./Data/Crop_Location_Data_20221201.csv")
display(crop_presence_data.head())
aux_data = pd.read_csv("./Data/S1DataVVProcessed.csv",index_col=0)
display(aux_data.head())

,Latitude and Longitude,Class of Land
0,"(10.323727047081501, 105.2516346045924)",Rice
1,"(10.322364360592521, 105.27843410554115)",Rice
2,"(10.321455902933202, 105.25254306225168)",Rice
3,"(10.324181275911162, 105.25118037576274)",Rice
4,"(10.324635504740822, 105.27389181724476)",Rice


,0,1,2,3,4,5,6,7,8,9,...,69,70,71,72,73,74,75,76,77,78
0,0.136713,0.019095,0.052516,0.113325,0.180918,0.084581,0.236054,0.138249,0.222160,0.082468,...,0.131975,0.116991,0.129285,0.034969,0.016335,0.118973,0.129160,0.072585,0.100532,0.147335
1,0.008367,0.073629,0.013057,0.102261,0.183774,0.112656,0.119053,0.125759,0.184625,0.052343,...,0.169769,0.093939,0.086115,0.061664,0.042575,0.174255,0.159475,0.112408,0.367571,0.253594
2,0.103548,0.036345,0.078220,0.217244,0.307188,0.277898,0.113407,0.082699,0.146055,0.026883,...,0.124293,0.082959,0.143766,0.017497,0.029017,0.096281,0.042791,0.049105,0.397772,0.136151
3,0.112445,0.039681,0.022097,0.074011,0.240549,0.296535,0.072409,0.135074,0.096405,0.059797,...,0.097248,0.079721,0.189882,0.030258,0.076895,0.081524,0.069514,0.099842,0.183957,0.230961
4,0.041582,0.012306,0.049128,0.104897,0.099337,0.077988,0.367231,0.664374,0.036919,0.331721,...,0.188886,0.175489,0.172606,0.058966,0.065165,0.099678,0.077754,0.125112,0.175304,0.111650


In [12]:
data=pd.concat([crop_presence_data,aux_data],axis=1)
data.drop(['Latitude and Longitude'],axis=1,inplace=True)
miss_rate=data.apply(lambda x: 1-x.isna().value_counts()[False]/len(x),axis=0)
# drop columns with missing rate > 0.5
data.drop(miss_rate[miss_rate>0.1].index,axis=1,inplace=True)
data.replace([-32768.000000],np.nan,inplace=True)
target_value=data['Class of Land'].apply(lambda x: 1 if x=='Rice' else 0)
data.drop(['Class of Land'],axis=1,inplace=True)
# fill missing values with mean
data.fillna(data.mean(axis=0),inplace=True)

In [13]:
data

,0,1,2,3,4,5,6,7,8,9,...,69,70,71,72,73,74,75,76,77,78
0,0.136713,0.019095,0.052516,0.113325,0.180918,0.084581,0.236054,0.138249,0.222160,0.082468,...,0.131975,0.116991,0.129285,0.034969,0.016335,0.118973,0.129160,0.072585,0.100532,0.147335
1,0.008367,0.073629,0.013057,0.102261,0.183774,0.112656,0.119053,0.125759,0.184625,0.052343,...,0.169769,0.093939,0.086115,0.061664,0.042575,0.174255,0.159475,0.112408,0.367571,0.253594
2,0.103548,0.036345,0.078220,0.217244,0.307188,0.277898,0.113407,0.082699,0.146055,0.026883,...,0.124293,0.082959,0.143766,0.017497,0.029017,0.096281,0.042791,0.049105,0.397772,0.136151
3,0.112445,0.039681,0.022097,0.074011,0.240549,0.296535,0.072409,0.135074,0.096405,0.059797,...,0.097248,0.079721,0.189882,0.030258,0.076895,0.081524,0.069514,0.099842,0.183957,0.230961
4,0.041582,0.012306,0.049128,0.104897,0.099337,0.077988,0.367231,0.664374,0.036919,0.331721,...,0.188886,0.175489,0.172606,0.058966,0.065165,0.099678,0.077754,0.125112,0.175304,0.111650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.209653,0.132859,0.340596,0.118031,0.212445,0.514146,0.202694,0.230627,0.553938,0.127156,...,0.255218,0.209444,0.477707,0.249642,0.190894,0.221262,0.152839,0.472225,0.193266,0.349118
596,0.345177,0.333710,0.156724,0.239881,0.354379,0.309487,0.100985,0.327562,0.577501,0.406944,...,0.459707,0.165649,0.225630,0.252992,0.276695,0.183418,0.426954,0.477477,0.174691,0.168521
597,0.422360,0.267347,0.946567,0.164242,0.374150,0.391451,0.318233,0.247475,0.396374,0.302028,...,0.376712,0.182609,0.460066,0.263639,0.376831,0.403079,0.604752,0.491092,0.293945,0.561207
598,0.088473,0.200898,0.239337,0.302809,0.287695,0.115140,0.175324,0.227792,0.389689,0.454797,...,0.258151,0.209155,0.120049,0.310898,0.306119,0.450686,0.223350,0.145055,0.340870,0.186654


In [14]:
# Loading Test Data
os.makedirs('./Submission_Data',exist_ok=True)
submit_fp ='./Submission_Data/'
submission_df = pd.read_csv('./Data/challenge_1_submission_template_correct_columns_fixed.csv')

replace_data,save_test_file = False,"./Data/TestS1DataVVProcessed.csv"
if not os.path.exists(save_test_file) or replace_data:
    # extract data
    pass
test_data = pd.read_csv(save_test_file)
test_data=test_data.loc[:,data.columns]
test_data.replace([-32768.000000],np.nan,inplace=True)
test_data.fillna(data.mean(axis=0),inplace=True)
X_test = test_data.values

##### Model Evaluation

In [15]:
# Tensorflow check cuda availability
devices_=tf.config.list_physical_devices('GPU')
# tf.config.set_logical_device_configuration(device=devices_[0],logical_devices=[tf.config.LogicalDeviceConfiguration(memory_limit=1024*22)])

In [16]:
# Sequenial Keras model 2 LSTM Layer with 10 neurons each and 1 Dense layer with 1 neuron
model = keras.Sequential()
model.add(layers.Input(shape=(79,1)))
model.add(layers.Bidirectional(layers.LSTM(50,return_sequences=True)))
model.add(layers.Bidirectional(layers.LSTM(50)))
model.add(layers.Dense(2, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(0.001), metrics=['accuracy'])


2023-03-26 09:51:27.124588: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-26 09:51:28.248130: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21267 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:3b:00.0, compute capability: 8.6
2023-03-26 09:51:28.249242: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22292 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:86:00.0, compute capability: 8.6


In [17]:
BATCH_SIZE=32
EPOCHS=100

In [19]:
# Split the data into train and test using sklearn random split
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(data, target_value, test_size=0.3, random_state=42)

# Convert the data into numpy array and reshape the data to fit the model
X_train = X_train.to_numpy().reshape(-1,79,1)
X_val = X_val.to_numpy().reshape(-1,79,1)
y_train = y_train.to_numpy().reshape(-1,1)
y_val = y_val.to_numpy().reshape(-1,1)

In [20]:
# Fit the model with model checkpoint best validation loss and early stopping with patience 5
checkpoint_filepath = './checkpoint/'
os.makedirs(os.path.dirname(checkpoint_filepath), exist_ok=True)
checkpoint_filepath = './checkpoint/checkpoint2'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
# model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1, validation_data=(X_val, y_val), callbacks=[model_checkpoint_callback,early_stopping])
model.fit(np.concatenate([X_train,X_val],axis=0), np.concatenate([y_train,y_val],axis=0), epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1,validation_split=0.2, callbacks=[model_checkpoint_callback])
model.load_weights(checkpoint_filepath)
# model.evaluate(x=X_val,y=y_val)


Epoch 1/100


2023-03-26 09:52:01.250982: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8201
2023-03-26 09:52:02.383308: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


15/15 [==============================] - 6s 108ms/step - loss: 0.6681 - accuracy: 0.4000 - val_loss: 0.6507 - val_accuracy: 0.3583
Epoch 2/100
15/15 [==============================] - 1s 37ms/step - loss: 0.5159 - accuracy: 0.7188 - val_loss: 0.4430 - val_accuracy: 0.8083
Epoch 3/100
15/15 [==============================] - 0s 33ms/step - loss: 0.4262 - accuracy: 0.7979 - val_loss: 0.7269 - val_accuracy: 0.7083
Epoch 4/100
15/15 [==============================] - 1s 37ms/step - loss: 0.5065 - accuracy: 0.7812 - val_loss: 0.2386 - val_accuracy: 0.9333
Epoch 5/100
15/15 [==============================] - 0s 33ms/step - loss: 0.2065 - accuracy: 0.9208 - val_loss: 1.1221 - val_accuracy: 0.8417
Epoch 6/100
15/15 [==============================] - 0s 33ms/step - loss: 0.6434 - accuracy: 0.8667 - val_loss: 0.3877 - val_accuracy: 0.8000
Epoch 7/100
15/15 [==============================] - 1s 37ms/step - loss: 0.3054 - accuracy: 0.8667 - val_loss: 0.2207 - val_accuracy: 0.9417
Epoch 8/100
15/15

In [21]:
model.evaluate(x=X_val,y=y_val)

6/6 [==============================] - 0s 12ms/step - loss: 0.0143 - accuracy: 1.0000


[0.01426310371607542, 1.0]

In [22]:
submission_df['target']=model.predict(X_test).argmax(axis=1)
submission_df['target']=submission_df['target'].apply(lambda x: 'Rice' if x==1 else 'Non Rice')

8/8 [==============================] - 1s 10ms/step


In [23]:
compare_df=pd.read_csv('./Data/Best_submission.csv').rename({'target':'best_target'},axis=1).merge(submission_df,on=['id'],how='left')
compare_df['is_equal']=compare_df.apply(lambda x: 1 if x['best_target']==x['target'] else 0,axis=1)

In [24]:
submission_df.to_csv(os.path.join(submit_fp,"Out2.csv"),index = False)

### Experiment 2: VH + VV Dataset

In [34]:
# Preprocessing Data Set

def data_preprocess(df_input):
    df = df_input.copy()
    miss_rate=df.apply(lambda x: 1-x.isna().value_counts()[False]/len(x),axis=0)
    df.drop(miss_rate[miss_rate>0.1].index,axis=1,inplace=True)
    df.replace([-32768.000000],np.nan,inplace=True)
    df.fillna(df.mean(axis=0),inplace=True)
    return df    

In [35]:
crop_presence_data = pd.read_csv("./Data/Crop_Location_Data_20221201.csv")
display(crop_presence_data.head())
vv_data = data_preprocess(pd.read_csv("./Data/S1DataVV.csv",index_col=0))
vh_data = data_preprocess(pd.read_csv("./Data/S1DataVH.csv",index_col=0))
display(vh_data.head())
display(vv_data.head())

,Latitude and Longitude,Class of Land
0,"(10.323727047081501, 105.2516346045924)",Rice
1,"(10.322364360592521, 105.27843410554115)",Rice
2,"(10.321455902933202, 105.25254306225168)",Rice
3,"(10.324181275911162, 105.25118037576274)",Rice
4,"(10.324635504740822, 105.27389181724476)",Rice


,0,1,2,3,4,5,6,7,8,9,...,69,70,71,72,73,74,75,76,77,78
0,0.004257,0.006852,0.009961,0.018752,0.056223,0.046123,0.028594,0.063402,0.063434,0.039090,...,0.022627,0.042624,0.056788,0.009047,0.010800,0.039736,0.025950,0.028481,0.035662,0.027236
1,0.005324,0.009781,0.003606,0.017472,0.023758,0.034844,0.136051,0.028345,0.046194,0.039960,...,0.066685,0.083743,0.111567,0.031366,0.052280,0.019166,0.018328,0.043713,0.020591,0.039421
2,0.012065,0.014525,0.069244,0.041848,0.033582,0.074039,0.104345,0.030171,0.042813,0.028010,...,0.032459,0.018907,0.083230,0.034847,0.010436,0.020277,0.017816,0.012061,0.020985,0.033677
3,0.004077,0.006702,0.007304,0.012286,0.074877,0.099660,0.019093,0.058226,0.042667,0.029065,...,0.043986,0.044804,0.086023,0.017242,0.019991,0.005546,0.017439,0.015310,0.013266,0.042812
4,0.001757,0.003051,0.002617,0.014432,0.009542,0.006758,0.041154,0.062224,0.010816,0.183996,...,0.042198,0.038811,0.036053,0.010171,0.019257,0.035105,0.022845,0.038898,0.009312,0.029217


,0,1,2,3,4,5,6,7,8,9,...,69,70,71,72,73,74,75,76,77,78
0,0.136713,0.019095,0.052516,0.113325,0.180918,0.084581,0.236054,0.138249,0.222160,0.082468,...,0.131975,0.116991,0.129285,0.034969,0.016335,0.118973,0.129160,0.072585,0.100532,0.147335
1,0.008367,0.073629,0.013057,0.102261,0.183774,0.112656,0.119053,0.125759,0.184625,0.052343,...,0.169769,0.093939,0.086115,0.061664,0.042575,0.174255,0.159475,0.112408,0.367571,0.253594
2,0.103548,0.036345,0.078220,0.217244,0.307188,0.277898,0.113407,0.082699,0.146055,0.026883,...,0.124293,0.082959,0.143766,0.017497,0.029017,0.096281,0.042791,0.049105,0.397772,0.136151
3,0.112445,0.039681,0.022097,0.074011,0.240549,0.296535,0.072409,0.135074,0.096405,0.059797,...,0.097248,0.079721,0.189882,0.030258,0.076895,0.081524,0.069514,0.099842,0.183957,0.230961
4,0.041582,0.012306,0.049128,0.104897,0.099337,0.077988,0.367231,0.664374,0.036919,0.331721,...,0.188886,0.175489,0.172606,0.058966,0.065165,0.099678,0.077754,0.125112,0.175304,0.111650


In [36]:
# Combine VV & VH Data such [ loc1[ date1[VV,VH],date2[VV,VH],...],loc2[ date1[VV,VH],date2[VV,VH],...],...]
vv_x=vv_data.apply(lambda x: [i for i in x.values],axis=1).tolist()
vh_y=vh_data.apply(lambda x: [i for i in x.values],axis=1).tolist()
print(len(vv_x),len(vh_y))

600 600


In [37]:
x=np.array([vv_x,vh_y])
x=x.transpose(1,2,0)

In [38]:
data =crop_presence_data.copy()
target_value=data['Class of Land'].apply(lambda x: 1 if x=='Rice' else 0)

In [39]:
# Loading Test Data
os.makedirs('./Submission_Data',exist_ok=True)
submit_fp ='./Submission_Data/'
submission_df = pd.read_csv('./Data/challenge_1_submission_template_correct_columns_fixed.csv')

replace_data,test_vv_fp,test_vh_fp = False,"./Data/TestS1DataVVProcessed.csv","./Data/TestS1DataVHProcessed.csv"
if not os.path.exists(test_vv_fp) or replace_data:
    # extract data
    pass
test_vv = pd.read_csv(test_vv_fp)
test_vh = pd.read_csv(test_vh_fp)
vv_x=test_vv.apply(lambda x: [i for i in x.values],axis=1).tolist()
vh_y=test_vh.apply(lambda x: [i for i in x.values],axis=1).tolist()
print(len(vv_x),len(vh_y))
X_test=np.array([vv_x,vh_y])
X_test=X_test.transpose(1,2,0)

250 250


##### Model Evaluation

In [40]:
# Tensorflow check cuda availability
devices_=tf.config.list_physical_devices('GPU')
# tf.config.set_logical_device_configuration(device=devices_[0],logical_devices=[tf.config.LogicalDeviceConfiguration(memory_limit=1024*22)])

In [41]:
# Sequenial Keras model 2 LSTM Layer with 10 neurons each and 1 Dense layer with 1 neuron
model = keras.Sequential()
model.add(layers.Input(shape=(79,2)))
model.add(layers.Bidirectional(layers.LSTM(30,return_sequences=True)))
model.add(layers.Bidirectional(layers.LSTM(30)))
model.add(layers.Dense(2, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.RMSprop(0.001), metrics=['accuracy'])


In [42]:
BATCH_SIZE=32
EPOCHS=100

In [43]:
# Split the data into train and test using sklearn random split
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(x, target_value, test_size=0.3, random_state=42)

# Convert the data into numpy array and reshape the data to fit the model
X_train = X_train.reshape(-1,79,2)
X_val = X_val.reshape(-1,79,2)
y_train = y_train.to_numpy().reshape(-1,1)
y_val = y_val.to_numpy().reshape(-1,1)

In [44]:
# Fit the model with model checkpoint best validation loss and early stopping with patience 5
checkpoint_filepath = './checkpoint/'
os.makedirs(os.path.dirname(checkpoint_filepath), exist_ok=True)
checkpoint_filepath = './checkpoint/checkpoint2'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
# model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1, validation_data=(X_val, y_val), callbacks=[model_checkpoint_callback,early_stopping])
model.fit(np.concatenate([X_train,X_val],axis=0), np.concatenate([y_train,y_val],axis=0), epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1,validation_split=0.2, callbacks=[model_checkpoint_callback])
model.load_weights(checkpoint_filepath)
model.evaluate(x=X_val,y=y_val)


Epoch 1/100
15/15 [==============================] - 5s 105ms/step - loss: 0.6696 - accuracy: 0.4771 - val_loss: 0.5809 - val_accuracy: 0.7500
Epoch 2/100
15/15 [==============================] - 1s 38ms/step - loss: 0.4763 - accuracy: 0.8125 - val_loss: 0.5117 - val_accuracy: 0.7667
Epoch 3/100
15/15 [==============================] - 1s 38ms/step - loss: 0.3753 - accuracy: 0.7958 - val_loss: 0.3565 - val_accuracy: 0.7750
Epoch 4/100
15/15 [==============================] - 1s 38ms/step - loss: 0.2523 - accuracy: 0.9062 - val_loss: 0.2168 - val_accuracy: 0.9333
Epoch 5/100
15/15 [==============================] - 1s 38ms/step - loss: 0.2435 - accuracy: 0.9271 - val_loss: 0.2460 - val_accuracy: 0.9417
Epoch 6/100
15/15 [==============================] - 1s 35ms/step - loss: 0.1837 - accuracy: 0.9542 - val_loss: 0.4227 - val_accuracy: 0.8583
Epoch 7/100
15/15 [==============================] - 1s 35ms/step - loss: 0.1717 - accuracy: 0.9458 - val_loss: 0.2240 - val_accuracy: 0.9333
Epoch

[0.02032529190182686, 1.0]

In [45]:
# model.save_weights('./checkpoint/Bilstm_best1.h5')

In [46]:
model.evaluate(x=X_val,y=y_val)

6/6 [==============================] - 0s 12ms/step - loss: 0.0203 - accuracy: 1.0000


[0.02032529190182686, 1.0]

In [47]:
submission_df['target']=model.predict(X_test).argmax(axis=1)
submission_df['target']=submission_df['target'].apply(lambda x: 'Rice' if x==1 else 'Non Rice')

8/8 [==============================] - 1s 11ms/step


In [50]:
compare_df=pd.read_csv('./Submission_Data/Best_Out_VV_VH_Bidirectional_LSTM98.csv').rename({'target':'best_target'},axis=1).merge(submission_df,on=['id'],how='left')
compare_df['is_equal']=compare_df.apply(lambda x: 1 if x['best_target']==x['target'] else 0,axis=1)

In [51]:
submission_df.to_csv(os.path.join(submit_fp,"Out_VV_VH_Bidirectional_LSTM.csv"),index = False)

### Experiment 3: RVI Value

In [32]:
# Preprocessing Data Set

def data_preprocess(df_input):
    df = df_input.copy()
    miss_rate=df.apply(lambda x: 1-x.isna().value_counts()[False]/len(x),axis=0)
    df.drop(miss_rate[miss_rate>0.1].index,axis=1,inplace=True)
    df.replace([-32768.000000],np.nan,inplace=True)
    df.fillna(df.mean(axis=0),inplace=True)
    return df    

In [33]:
crop_presence_data = pd.read_csv("./Data/Crop_Location_Data_20221201.csv")
display(crop_presence_data.head())
rvi_data = data_preprocess(pd.read_csv("./Data/Raw RVI values.csv"))
display(rvi_data.head())

,Latitude and Longitude,Class of Land
0,"(10.323727047081501, 105.2516346045924)",Rice
1,"(10.322364360592521, 105.27843410554115)",Rice
2,"(10.321455902933202, 105.25254306225168)",Rice
3,"(10.324181275911162, 105.25118037576274)",Rice
4,"(10.324635504740822, 105.27389181724476)",Rice


,0,1,2,3,4,5,6,7,8,9,...,75,76,77,78,79,80,81,82,83,84
0,1.268812,1.177804,1.078746,0.885944,1.011423,0.576550,0.825536,1.215215,0.617741,0.730946,...,0.805856,0.945279,0.683113,0.651854,0.615287,0.616962,0.599794,0.790055,0.555535,0.205384
1,1.386990,0.986378,1.056098,1.269690,1.002210,0.828510,0.486269,0.791543,0.898176,0.586530,...,1.029705,0.799573,1.248745,0.693501,0.635611,0.494811,0.697652,0.331825,0.200730,0.691492
2,0.853811,1.424612,0.946962,0.856900,0.654258,0.842964,0.734223,1.134528,0.625878,0.831439,...,0.751218,0.947396,0.798228,0.825979,0.471235,0.610515,0.566653,0.367716,0.291665,0.230669
3,1.026404,1.012573,0.563292,0.870734,0.808345,0.884417,0.567212,0.828071,0.510699,0.883200,...,0.822316,0.957821,0.953764,0.751689,0.669149,0.490925,0.780768,0.498302,0.428138,0.259073
4,1.117442,0.664129,0.556181,0.800441,0.853133,0.769346,0.603781,0.777530,0.610921,1.151756,...,0.585791,0.458077,0.514433,0.490509,0.476429,0.522104,0.290942,0.519854,0.452367,0.200230


In [34]:
# Combine VV & VH Data such [ loc1[ date1[VV,VH],date2[VV,VH],...],loc2[ date1[VV,VH],date2[VV,VH],...],...]
rvi_x=rvi_data.apply(lambda x: [i for i in x.values],axis=1).tolist()
print(len(rvi_x))

600


In [35]:
x=np.array([rvi_x]).reshape(-1,len(rvi_x[0]),1)

In [36]:
data =crop_presence_data.copy()
target_value=data['Class of Land'].apply(lambda x: 1 if x=='Rice' else 0)

In [37]:
# Loading Test Data
os.makedirs('./Submission_Data',exist_ok=True)
submit_fp ='./Submission_Data/'
submission_df = pd.read_csv('./Data/challenge_1_submission_template_correct_columns_fixed.csv')

replace_data,test_vv_fp = False,"./Data/Raw RVI test values.csv",
if not os.path.exists(test_vv_fp) or replace_data:
    # extract data
    pass
test_vv = pd.read_csv(test_vv_fp)
rvi_x=test_vv.apply(lambda x: [i for i in x.values],axis=1).tolist()
print(len(rvi_x))
X_test=np.array(rvi_x).reshape(-1,len(rvi_x[0]),1)

250


##### Model Evaluation

In [38]:
# Tensorflow check cuda availability
devices_=tf.config.list_physical_devices('GPU')
# tf.config.set_logical_device_configuration(device=devices_[0],logical_devices=[tf.config.LogicalDeviceConfiguration(memory_limit=1024*22)])

In [39]:
x.shape

(600, 85, 1)

In [48]:
# Sequenial Keras model 2 LSTM Layer with 10 neurons each and 1 Dense layer with 1 neuron
model = keras.Sequential()
model.add(layers.Input(shape=(x.shape[1],1)))
model.add(layers.LSTM(10,return_sequences=True))
model.add(layers.LSTM(10))
model.add(layers.Dense(2, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(0.001), metrics=['accuracy'])


In [49]:
BATCH_SIZE=32
EPOCHS=100

In [50]:
# Split the data into train and test using sklearn random split
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(x, target_value, test_size=0.3, random_state=42)

# Convert the data into numpy array and reshape the data to fit the model
y_train = y_train.to_numpy().reshape(-1,1)
y_val = y_val.to_numpy().reshape(-1,1)

In [51]:
# Fit the model with model checkpoint best validation loss and early stopping with patience 5
checkpoint_filepath = './checkpoint/'
os.makedirs(os.path.dirname(checkpoint_filepath), exist_ok=True)
checkpoint_filepath = './checkpoint/checkpoint2'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
# model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1, validation_data=(X_val, y_val), callbacks=[model_checkpoint_callback,early_stopping])
model.fit(np.concatenate([X_train,X_val],axis=0), np.concatenate([y_train,y_val],axis=0), epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1,validation_split=0.2, callbacks=[model_checkpoint_callback,early_stopping])
# model.load_weights(checkpoint_filepath)
# model.evaluate(x=X_val,y=y_val)


Epoch 1/100
15/15 [==============================] - 3s 76ms/step - loss: 0.7034 - accuracy: 0.4292 - val_loss: 0.6973 - val_accuracy: 0.4750
Epoch 2/100
15/15 [==============================] - 0s 19ms/step - loss: 0.6936 - accuracy: 0.5063 - val_loss: 0.6935 - val_accuracy: 0.4750
Epoch 3/100
15/15 [==============================] - 0s 20ms/step - loss: 0.6883 - accuracy: 0.5063 - val_loss: 0.6903 - val_accuracy: 0.4750
Epoch 4/100
15/15 [==============================] - 0s 19ms/step - loss: 0.6834 - accuracy: 0.5063 - val_loss: 0.6843 - val_accuracy: 0.4750
Epoch 5/100
15/15 [==============================] - 0s 22ms/step - loss: 0.6763 - accuracy: 0.5833 - val_loss: 0.6749 - val_accuracy: 0.6083
Epoch 6/100
15/15 [==============================] - 0s 19ms/step - loss: 0.6670 - accuracy: 0.5562 - val_loss: 0.6685 - val_accuracy: 0.4917
Epoch 7/100
15/15 [==============================] - 0s 22ms/step - loss: 0.6556 - accuracy: 0.6542 - val_loss: 0.6504 - val_accuracy: 0.7083
Epoch 

In [52]:
model.evaluate(x=X_val,y=y_val)

6/6 [==============================] - 0s 7ms/step - loss: 6.0339e-04 - accuracy: 1.0000


[0.0006033917306922376, 1.0]

In [53]:
submission_df['target']=model.predict(X_test).argmax(axis=1)
submission_df['target']=submission_df['target'].apply(lambda x: 'Rice' if x==1 else 'Non Rice')

8/8 [==============================] - 1s 8ms/step


In [54]:
compare_df=pd.read_csv('./Data/Best_submission.csv').rename({'target':'best_target'},axis=1).merge(submission_df,on=['id'],how='left')
compare_df['is_equal']=compare_df.apply(lambda x: 1 if x['best_target']==x['target'] else 0,axis=1)

In [47]:
submission_df.to_csv(os.path.join(submit_fp,"Out_RVI.csv"),index = False)